## Dependencies

In [ ]:
!pip install weaviate-client

In [ ]:
!pip install --pre -I "weaviate-client==4.3.b2"

## Connect to Weaviate

In [ ]:
import weaviate, os

cohere_key = os.environ["COHERE_API_KEY"] # Replace with your Cohere key

# Connect to your local Weaviate instance deployed with Docker
client = weaviate.connect_to_local(
  headers={
    "X-COHERE-Api-Key": cohere_key
  }
)

# Option 2
# Connect to your Weaviate Client Service cluster
# client = weaviate.connect_to_wcs(
#     cluster_id="WCS-CLUSTER-ID", # Replace with your WCS cluster ID
#     auth_credentials=weaviate.AuthApiKey(
#       api_key="WCS-API-KEY" # Replace with your WCS API KEY
#     ),
#     headers={
#       "X-COHERE-Api-Key": cohere_key
#     }
# )

client.is_ready()

## Create a collection
> Collection stores your data and vector embeddings.

In [ ]:
# Note: in practice, you shouldn't rerun this cell, as it deletes your data
# in "JeopardyQuestion", and then you need to re-import it again.
import weaviate.classes as wvc

# Delete the collection if it already exists
if (client.collections.exists("JeopardyQuestion")):
    client.collections.delete("JeopardyQuestion")

client.collections.create(
    name="JeopardyQuestion",

    vectorizer_config=wvc.Configure.Vectorizer.text2vec_cohere( # specify the vectorizer and model type you're using
        model="embed-multilingual-v2.0", # defaults to embed-multilingual-v2.0 if not set
        truncate="RIGHT", # defaults to RIGHT if not set 
    ),

    properties=[ # defining properties (data schema) is optional
        wvc.Property(name="Question", data_type=wvc.DataType.TEXT), 
        wvc.Property(name="Answer", data_type=wvc.DataType.TEXT),
        wvc.Property(name="Category", data_type=wvc.DataType.TEXT, skip_vectorization=True), 
    ]
)

print("Successfully created collection: JeopardyQuestion.")

## Import the Data

In [ ]:
import requests, json
url = 'https://raw.githubusercontent.com/weaviate/weaviate-examples/main/jeopardy_small_dataset/jeopardy_tiny.json'
resp = requests.get(url)
data = json.loads(resp.text)

# Get a collection object for "JeopardyQuestion"
jeopardy = client.collections.get("JeopardyQuestion")

# Insert data objects
response = jeopardy.data.insert_many(data)

# Note, the `data` array contains 10 objects, which is great to call insert_many with.
# However, if you have a milion objects to insert, then you should spit them into smaller batches (i.e. 100-1000 per insert)

if (response.has_errors):
    print(response.errors)
else:
    print("Insert complete.")

## Hybrid Search

The `alpha` parameter determines the weight given to the sparse and dense search methods. `alpha = 0` is pure sparse (bm25) search, whereas `alpha = 1` is pure dense (vector) search. 

Alpha is an optional parameter. The default is set to `0.75`.

### Hybrid Search only

The below query is finding Jeopardy questions about animals and is limiting the output to only two results. Notice `alpha` is set to `0.80`, which means it is weighing the vector search results more than bm25. If you were to set `alpha = 0.25`, you would get different results. 

In [ ]:
# note, you can reuse the collection object from the previous cell.
# Get a collection object for "JeopardyQuestion"
jeopardy = client.collections.get("JeopardyQuestion")

response = jeopardy.query.hybrid(
    query="northern beast",
    alpha=0.8,
    limit=3
)

for item in response.objects:
    print("ID:", item.uuid)
    print("Data:", json.dumps(item.properties, indent=2), "\n")

### Hybrid Search on a specific property

The `properties` parameter allows you to list the properties that you want bm25 to search on.

In [ ]:
response = jeopardy.query.hybrid(
    query="northern beast",
    query_properties=["question"],
    alpha=0.8,
    limit=3
)

for item in response.objects:
    print("ID:", item.uuid)
    print("Data:", json.dumps(item.properties, indent=2), "\n")

### Hybrid Search with a `where` filter

Find Jeopardy questions about elephants, where the category is set to Animals.

In [ ]:
import weaviate.classes as wvc # wvc is an alias to save us from typing weaviate.classes everywhere ;)

response = jeopardy.query.hybrid(
    query="northern beast",
    alpha=0.8,
    filters=wvc.Filter("category").equal("Animals"),
    limit=3
)

for item in response.objects:
    print("ID:", item.uuid)
    print("Data:", json.dumps(item.properties, indent=2), "\n")

### Hybrid Search with a custom vector

You can pass in your own vector as input into the hybrid query, by using the `vector` parameter. 

In [ ]:
vector = [0.17392578125, -0.2024509429931640625, 0.330859375, 0.439111328125, 0.5478515625, 0.22216796875, 0.293701171875, -0.26806640625, 0.11944580078125, 0.28759765625, -0.1005859375, -0.29638671875, -0.380615234375, -0.1773681640625, -0.0275421142578125, -0.006099700927734375, 0.53369140625, -0.03900146484375, -0.265380859375, 0.230224609375, -0.10028076171875, 0.328125, 0.239990234375, -0.34521484375, 0.09918212890625, 0.1260986328125, -0.347900390625, 0.189697265625, -0.176025390625, 0.0362548828125, -0.02581787109375, -0.0699462890625, -0.044677734375, 0.48291015625, -0.27490234375, 0.345458984375, 0.177001953125, 0.35791015625, 0.438232421875, 0.246826171875, 0.284912109375, 0.265380859375, -0.0002658367156982422, -0.017242431640625, 0.198486328125, -0.062286376953125, -0.18408203125, 0.274658203125, -0.57763671875, 0.01214599609375, -0.0628662109375, -0.044097900390625, -0.253662109375, -0.2276611328125, -0.344970703125, 0.39404296875, 0.16845703125, 0.394287109375, 0.032806396484375, -0.281005859375, 0.218017578125, -0.0462646484375, -0.261962890625, -0.320068359375, 0.86669921875, 0.2073974609375, 0.05419921875, 0.51513671875, -0.09539794921875, 0.469970703125, -0.085205078125, 0.53564453125, 0.681640625, 0.0859375, -0.04547119140625, -0.1075439453125, 0.0465087890625, -0.265625, 0.390380859375, -0.279541015625, 0.6201171875, 0.11273193359375, -0.22900390625, 0.32666015625, -0.407958984375, 0.35205078125, -0.12646484375, 0.311279296875, -0.043060302734375, 0.40673828125, -0.0401611328125, 0.2203369140625, 0.03106689453125, 0.0247802734375, 0.1275634765625, 0.295166015625, 0.15478515625, -0.1446533203125, 0.468994140625, -0.156494140625, 0.2252197265625, -0.1864013671875, -0.416015625, 0.215576171875, 0.09710693359375, -0.357177734375, -0.166015625, -0.021759033203125, 0.224853515625, 0.2081298828125, 0.345947265625, -0.10968017578125, -0.1280517578125, -0.073974609375, 0.439453125, -0.125244140625, 0.287841796875, 0.1339111328125, 0.30029296875, -1.248046875, 0.36376953125, 0.1517333984375, -0.329833984375, -0.10821533203125, -0.354736328125, 0.061920166015625, 0.51123046875, -0.08453369140625, 0.669921875, 0.402587890625, -0.01340484619140625, 0.212158203125, 0.159423828125, 0.716796875, 0.41455078125, 0.51416015625, 0.441162109375, 0.0081329345703125, 0.077392578125, -0.1707763671875, -0.220458984375, -0.327880859375, 0.11004638671875, 0.703125, -0.040618896484375, 0.2626953125, 0.10760498046875, 0.31982421875, -0.06488037109375, 0.2152099609375, -0.1915283203125, -0.122802734375, 0.03753662109375, 0.48486328125, -0.326904296875, 0.0006060600280761719, 0.16943359375, -0.10784912109375, 0.128662109375, -0.146728515625, 0.7255859375, 0.3251953125, -0.178955078125, -0.406005859375, -0.140380859375, 0.515625, 0.0535888671875, 0.6044921875, 0.5576171875, -0.270751953125, -0.127685546875, 0.1551513671875, 0.106201171875, -0.272705078125, 0.25146484375, 0.402099609375, 0.0164947509765625, 0.1407470703125, 0.0435791015625, -0.1375732421875, 0.047576904296875, 0.51513671875, 0.058990478515625, 0.0215606689453125, 0.69580078125, 0.5068359375, 0.1595458984375, -0.01526641845703125, -0.261962890625, -0.229248046875, 0.39697265625, 0.061676025390625, 0.59130859375, 0.0870361328125, -0.1634521484375, 0.051910400390625, 0.10675048828125, 0.51171875, -0.290283203125, 0.264892578125, 0.196044921875, -0.1492919921875, -0.27197265625, -0.214599609375, 0.4609375, -0.22412109375, 0.178955078125, 0.1063232421875, -0.15625, -0.310546875, 0.09979248046875, 0.056610107421875, 0.465087890625, 0.194091796875, -0.37646484375, 0.0816650390625, -0.69140625, -0.27783203125, 0.408935546875, -0.003772735595703125, -0.230224609375, 0.499267578125, 0.53515625, 0.3173828125, 0.06048583984375, -0.168701171875, -0.1732177734375, -0.475830078125, 0.11773681640625, 0.3203125, 0.1265869140625, 0.49609375, -0.043487548828125, 0.042633056640625, 0.2412109375, 0.4052734375, 0.365234375, 0.3193359375, -0.0292205810546875, 0.13818359375, 0.1553955078125, -0.1656494140625, -0.372802734375, 0.7294921875, 0.3935546875, -0.335205078125, 0.06134033203125, 0.07305908203125, -0.0009698867797851562, -0.27587890625, -0.0179443359375, -0.2410888671875, 0.0086517333984375, 0.22802734375, -0.2132568359375, 0.1834716796875, 0.048187255859375, -0.177490234375, 0.426025390625, 0.07769775390625, -0.051239013671875, 0.1490478515625, 0.489013671875, 0.481201171875, -0.0193939208984375, -0.28662109375, -0.07672119140625, 0.681640625, 0.197509765625, 0.4970703125, 0.33642578125, 0.025726318359375, 0.1807861328125, 0.1416015625, 0.00489044189453125, -0.127685546875, 0.240478515625, -0.07098388671875, -0.490966796875, 0.23681640625, 0.57666015625, -0.09710693359375, -0.249755859375, -0.026031494140625, 0.09002685546875, 0.10736083984375, -0.1636962890625, -0.04254150390625, -0.429931640625, 0.10321044921875, 0.269287109375, 0.50439453125, 0.0211944580078125, -0.1282958984375, 0.163330078125, 0.0875244140625, 0.0322265625, 0.2509765625, 0.26220703125, 0.1824951171875, 0.7119140625, -0.46533203125, 0.63037109375, 0.4072265625, -0.0550537109375, -0.176025390625, -0.127685546875, 0.2392578125, -0.278076171875, 0.340087890625, 0.62939453125, -0.263671875, -0.16943359375, 0.07672119140625, 0.240478515625, 0.03167724609375, -0.038330078125, 0.10711669921875, 0.7333984375, 0.059051513671875, 0.0379638671875, -0.185791015625, -0.37548828125, -0.055755615234375, 0.2410888671875, -0.56982421875, 0.28173828125, -0.05865478515625, 0.110107421875, -0.05889892578125, 0.4248046875, 0.10260009765625, -0.430908203125, 0.166015625, -0.154296875, 0.13037109375, -0.094970703125, 0.06005859375, 0.090576171875, -0.194091796875, 0.09613037109375, -0.0911865234375, 0.21875, 0.486572265625, -0.09246826171875, 0.2108154296875, 0.403564453125, -0.2144775390625, 0.48974609375, -0.01904296875, 0.13671875, -0.103759765625, -0.166015625, -0.270263671875, 0.53662109375, -0.185546875, 0.0158538818359375, -0.11865234375, 0.06903076171875, -0.058837890625, 0.235595703125, 0.03753662109375, 0.07275390625, -0.15380859375, 0.26953125, 0.230224609375, 0.1943359375, -0.10546875, 0.12646484375, 0.08905029296875, 0.146484375, -0.270263671875, -0.2484130859375, -0.13232421875, -0.517578125, -0.479248046875, -0.1656494140625, 0.52587890625, 0.5712890625, 0.10986328125, 0.168701171875, 0.6162109375, 0.02520751953125, -0.08026123046875, -0.331298828125, -0.20654296875, 0.08380126953125, -0.10345458984375, 0.0379638671875, -0.286865234375, -0.057891845703125, -0.157958984375, 0.2088623046875, 0.421142578125, -0.08984375, 0.17431640625, 0.120849609375, 0.1588134765625, 0.209228515625, 0.18310546875, 0.004047393798828125, -0.01837158203125, 0.5048828125, 0.14697265625, 4.2578125, 0.0919189453125, 0.27880859375, 0.09051513671875, -0.2205810546875, 0.355712890625, 0.293212890625, -0.157958984375, 0.05865478515625, 0.337158203125, -0.147216796875, 0.072998046875, -0.07220458984375, 0.0269317626953125, -0.1878662109375, 0.455322265625, 0.87548828125, 0.01873779296875, 0.1656494140625, 0.353759765625, -0.447265625, 0.4150390625, 0.41162109375, 0.028045654296875, 0.6787109375, 0.0267333984375, 0.487548828125, -0.1004638671875, 0.0054168701171875, 0.12841796875, 0.200439453125, -0.08477783203125, 0.234375, 0.19189453125, -0.49365234375, 0.358642578125, 0.349853515625, 0.08282470703125, -0.054443359375, 0.1142578125, -0.136474609375, 0.2440185546875, 0.1563720703125, 0.457275390625, 0.01313018798828125, -0.322265625, 0.3388671875, 0.62939453125, 0.1490478515625, 0.0390625, 0.0909423828125, 0.00852203369140625, -0.029541015625, 0.025390625, 0.419677734375, 0.43994140625, 0.2156982421875, 0.28369140625, -0.10870361328125, 0.137451171875, 0.0733642578125, -0.2255859375, 0.62744140625, -0.238525390625, -0.2191162109375, 0.09185791015625, 0.43798828125, 0.36572265625, 0.061859130859375, -0.414794921875, 0.445556640625, 0.31689453125, 0.2113037109375, -0.138916015625, -0.0687255859375, -0.114013671875, -0.498046875, 0.320556640625, 0.07720947265625, 0.0758056640625, -0.09234619140625, -0.1763916015625, 0.130859375, 0.159423828125, -0.1932373046875, 0.5458984375, -0.052947998046875, -0.3623046875, 0.448486328125, 0.10675048828125, 0.259765625, -0.10906982421875, 0.2059326171875, -0.11663818359375, 0.033782958984375, 0.0657958984375, 0.07537841796875, -3.9765625, -0.023345947265625, 0.07916259765625, -0.1719970703125, 0.1497802734375, -0.1199951171875, 0.0784912109375, 0.60546875, -0.7060546875, 0.480224609375, -0.26611328125, 0.4169921875, -0.412353515625, 0.5126953125, -0.07794189453125, 0.1029052734375, 0.1815185546875, -0.0550537109375, 0.267822265625, -0.26318359375, -0.1983642578125, 0.2425537109375, 0.359619140625, -0.302978515625, 0.06256103515625, 0.0229034423828125, -0.0172576904296875, -0.50634765625, -0.212890625, -0.1063232421875, 0.33056640625, -0.281982421875, 0.5458984375, -0.281982421875, -0.2724609375, -0.04510498046875, 0.52734375, -0.26708984375, 0.392578125, 0.414794921875, -0.09515380859375, 0.16552734375, -0.043121337890625, -0.0318603515625, -0.01052093505859375, 0.1448974609375, -0.08599853515625, -0.0008521080017089844, -0.330810546875, 0.32275390625, 0.1566162109375, 0.135498046875, -0.09967041015625, -0.1365966796875, 0.441162109375, -0.1571044921875, 0.386474609375, 0.1278076171875, 0.10089111328125, 0.0963134765625, -0.0285186767578125, -0.23388671875, 0.0034198760986328125, -0.111328125, 0.193359375, -0.0130157470703125, -0.083251953125, 0.359130859375, 0.445068359375, 0.1553955078125, -0.212646484375, 0.03851318359375, 0.016845703125, -0.0711669921875, 0.04217529296875, 0.04241943359375, 0.03411865234375, -0.15380859375, 0.65966796875, 0.1207275390625, -0.056610107421875, -0.0997314453125, -0.354248046875, 0.06304931640625, 2.830078125, 0.297119140625, 2.07421875, 0.1046142578125, -0.53076171875, 0.447998046875, -0.468994140625, 0.1754150390625, 0.0308990478515625, 0.1583251953125, 0.28857421875, 0.352294921875, -0.31396484375, 0.10382080078125, -0.330810546875, 0.3291015625, 0.392578125, -0.384765625, -0.08349609375, -0.432861328125, 0.38427734375, -0.007640838623046875, -0.1192626953125, 0.35302734375, 0.420654296875, -0.025787353515625, -0.08343505859375, 0.1470947265625, 0.049072265625, -0.295166015625, 0.335693359375, -0.039947509765625, 0.1729736328125, 0.27001953125, 0.1392822265625, 0.2388916015625, 0.0034275054931640625, 4.68359375, 0.2332763671875, -0.221923828125, 0.08135986328125, -0.043670654296875, 0.0305023193359375, 0.1976318359375, -0.0251312255859375, 0.1947021484375, 0.3818359375, 0.486083984375, -0.0458984375, 0.1529541015625, -0.1280517578125, 0.1622314453125, 0.141357421875, 0.146484375, 0.265625, 0.26318359375, -0.04095458984375, 0.08203125, 0.043548583984375, 0.451416015625, -0.218505859375, 0.1829833984375, 0.08343505859375, 0.30859375, -0.2222900390625, -0.1270751953125, 0.346435546875, -0.1309814453125, 5.40625, 0.083251953125, 0.046539306640625, -0.1751708984375, 0.0218353271484375, 0.2388916015625, -0.06207275390625, -0.638671875, -0.477294921875, -0.1357421875, -0.07696533203125, 0.250732421875, -0.25, 0.59765625, 0.206298828125, 0.08428955078125, -0.1812744140625, 0.1646728515625, -0.0662841796875, 0.204833984375, 0.47998046875, -0.044097900390625, 0.04248046875, -0.25341796875, -0.02398681640625, -0.650390625, -0.0386962890625, 0.325927734375, -0.316650390625, -0.1778564453125, 0.27783203125, 0.2041015625, -0.2291259765625, 0.81396484375, -0.53466796875, -0.399658203125, 0.405517578125, 0.243408203125, -0.050872802734375, 0.04754638671875, 0.12261962890625, 0.463623046875, 0.1566162109375, -0.00995635986328125, -0.11273193359375, -0.044097900390625, -0.04296875, -0.0732421875, -0.09674072265625, 0.06781005859375, 0.006000518798828125, -0.08404541015625, 0.759765625, 0.30029296875, 0.310302734375, 0.5439453125, 0.44970703125, 0.1744384765625, 0.09375, -0.057281494140625, 0.42138671875, 0.1026611328125, -0.0013370513916015625, 0.69677734375, 0.351806640625, 0.263916015625, 0.395751953125, 0.09967041015625, 0.65185546875, -0.053680419921875, -0.01171875, 0.0745849609375, 0.134765625, 0.169677734375, -0.143310546875, 0.404296875, 0.02703857421875, -0.2235107421875, 0.311767578125, -0.256591796875, -0.209716796875, 0.04852294921875, -0.212890625, -0.343505859375, 0.038421630859375, 0.30029296875, 0.018524169921875, 0.362548828125, 0.1949462890625, 0.05853271484375, -0.09454345703125, 0.23828125, 0.0478515625, 0.39794921875, 0.059906005859375, -0.05535888671875, 0.449462890625, 0.40478515625, 0.3779296875, 0.275390625, -0.06597900390625, 0.14208984375, 0.0255279541015625, -0.1673583984375, 0.252197265625, 0.053924560546875, 0.008270263671875, 0.1507568359375, 0.0399169921875, 0.039398193359375, 0.361328125, 0.423583984375, -0.1357421875, 0.11907958984375, 0.0031909942626953125]

response = jeopardy.query.hybrid(
    query="animal",
    vector=vector,
    limit=2
)

for item in response.objects:
    print("ID:", item.uuid)
    print("Data:", json.dumps(item.properties, indent=2), "\n")